# Free swelling using geometry

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from ngsolve import *
from netgen.csg import *
from ngsolve.webgui import Draw

In [2]:
# Geometry of 1/8 
L = 90
d = 23.5
L3 = 3.0

left  = Plane (Pnt(0,0,0), Vec(-1,0,0) ).bc('left')
right = Plane (Pnt(0.5*L,0,0), Vec( 1,0,0) )
bot = Plane (Pnt(0,0,0), Vec(0,-1,0) ).bc('bot')
top  = Plane (Pnt(0,0.5*d,0), Vec(0, 1,0) )
back   = Plane (Pnt(0,0,0), Vec(0,0,-1) ).bc('back')
front   = Plane (Pnt(0,0,0.5*L3), Vec(0,0, 1) )

brick = left * right * front * back * bot * top

geo = CSGeometry()
geo.Add (brick)
mesh = Mesh(geo.GenerateMesh(maxh=1))
Draw(mesh)
print(mesh.GetBoundaries())

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

('left', 'bot', 'default', 'default', 'back', 'default')


# Parameters

In [3]:
global phi0
#phi0 = 0.2035 # or 0.3
phi0 = 0.3

global chi 
chi = 0.45

global entropic_unit
entropic_unit=136.6   #This is (k_B * T)/(V_m), measured in Megapascals

global G    #It is the shear modulus. Will be defined in the next script (next jupy cell).

phi = lambda J: phi0/J
dH = lambda J: (1-1/1000) * phi(J) + np.log(1-phi(J)) + chi * phi(J)**2

#gammafun = lambda lamb: -dH(lamb)/lamb # bonded
gammafun = lambda lamb: -dH(lamb**3)*lamb

lambda_target = 1.49
gamma_target = gammafun(lambda_target)
G_target = gamma_target*entropic_unit
print("gamma target:", gamma_target)
print("shear modulus target:", G_target)

print("lo que tiene que crecer:", d*(lambda_target-1))
G=G_target
print( "desplazamineto esperados:", (lambda_target-1)/2*np.array([L,d,L3]) )
#v = np.linspace(1,2,100)
#plt.plot(v,gammafun(v)*entropic_unit)

gamma target: 0.001145522071527073
shear modulus target: 0.15647831497059816
lo que tiene que crecer: 11.515
desplazamineto esperados: [22.05    5.7575  0.735 ]


In [4]:
def F(u):
    return Id(mesh.dim) + Grad(u)

global gamma
gamma = G/entropic_unit
print(gamma)

def Gels (F):
    J = Det(F)
    phi = phi0/J
    H = (J - phi0)*log(1-phi)  + phi0 * chi*(1-phi) + phi0/1000*log(phi)
    C = F.trans * F
    return 0.5*gamma*(Trace(C)) + H

0.001145522071527073


In [5]:
# Finite element space with slipp boundary conditions on back|bot|left
V = VectorH1(mesh, order=2, dirichletx = 'left', dirichlety='bot', dirichletz='back')

# Construction of bilinear form
u  = V.TrialFunction()
I = Id(mesh.dim)
F = I + Grad(u)
a = BilinearForm(V, symmetric=True)

# hydrogel model
a += Variation(  Gels (F).Compile() * dx)

In [6]:
def SolveNonlinearMinProblem(a,gfu,tol=1e-08,maxits=250, scenes=None):
    res = gfu.vec.CreateVector()
    du  = gfu.vec.CreateVector()

    for it in range(maxits):
        print ("Newton iteration {:3}".format(it),end=", ")
        print ("energy = {:16}".format(a.Energy(gfu.vec)),end="")
        #solve linearized problem:
        a.Apply (gfu.vec, res)
        a.AssembleLinearization (gfu.vec)
        inv = a.mat.Inverse(V.FreeDofs())
        alpha = 5e-1
        du.data = alpha * inv * res

        #update iteration
        gfu.vec.data -= du

        #stopping criteria
        stopcritval = sqrt(abs(InnerProduct(du,res)))
        print ("<A u",it,", A u",it,">_{-1}^0.5 = ", stopcritval)
        if stopcritval < tol:
            break
        for sc in scenes:
            sc.Redraw()

In [7]:
# Solve

gfu = GridFunction(V)
gfu.vec[:] = 0

scene = Draw (gfu, mesh, "u", deformation=True)

# scene0 = Draw (gfu.components[0], mesh, "u", deformation=True)
# scene1 = Draw (gfu.components[1], mesh, "u", deformation=True)

# scene2 = Draw (gfu.components[2], mesh, "u", deformation=True)

SetVisualization (deformation=True)

res = gfu.vec.CreateVector()
du = gfu.vec.CreateVector()

# scenes = [scene, scene0, scene1, scene2]
scenes = [scene]
SolveNonlinearMinProblem(a,gfu, scenes=scenes)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

Newton iteration   0, energy = -121.99481492764421<A u 0 , A u 0 >_{-1}^0.5 =  1.5812555297125008
Newton iteration   1, energy = -123.9735199456669<A u 1 , A u 1 >_{-1}^0.5 =  1.2859745704927212
Newton iteration   2, energy = -125.27881870605482<A u 2 , A u 2 >_{-1}^0.5 =  1.0204961804832948
Newton iteration   3, energy = -126.09747529045966<A u 3 , A u 3 >_{-1}^0.5 =  0.7795333522749873
Newton iteration   4, energy = -126.57231306714276<A u 4 , A u 4 >_{-1}^0.5 =  0.563346992207912
Newton iteration   5, energy = -126.81834955340773<A u 5 , A u 5 >_{-1}^0.5 =  0.3788804035518878
Newton iteration   6, energy = -126.92865455723872<A u 6 , A u 6 >_{-1}^0.5 =  0.23527236168219284
Newton iteration   7, energy = -126.97083701861554<A u 7 , A u 7 >_{-1}^0.5 =  0.1356749575902735
Newton iteration   8, energy = -126.98477507277238<A u 8 , A u 8 >_{-1}^0.5 =  0.07388725225427774
Newton iteration   9, energy = -126.98889132710426

KeyboardInterrupt: 

In [8]:
# VTKOutput object
vtk = VTKOutput(ma=mesh,
                coefs=[gfu],
                names = ["displacement"],
                filename="result_free_swelling_sub3",
                subdivision=3)
# Exporting the results:
vtk.Do()

'result_free_swelling_sub3'

In [9]:
from RUN_Gel_free_swelling import gel_free_swelling, Solve_gel_free_swelling
GEL = gel_free_swelling()
GEL.geometry()

modelling = Solve_gel_free_swelling(GEL)
modelling.Solve()
Draw(modelling.gfu)

ModuleNotFoundError: No module named 'RUN_Gel_free_swelling'